In [ ]:
# Imports 
import numpy as np
import random

from datetime import datetime

# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3


In [ ]:
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Berlin': [74, 63],
 'London': [23, 51],
 'Moscow': [94, 23],
 'Barcelona': [13, 29],
 'Rome': [61, 22],
 'Paris': [82, 69],
 'Vienna': [98, 49],
 'Munich': [22, 53],
 'Istanbul': [8, 59],
 'Kyiv': [71, 11],
 'Bucharest': [31, 72],
 'Minsk': [70, 68],
 'Warsaw': [37, 91],
 'Budapest': [11, 56],
 'Milan': [37, 17],
 'Prague': [90, 89],
 'Sofia': [73, 74],
 'Birmingham': [72, 60],
 'Brussels': [9, 41],
 'Amsterdam': [24, 66]}

In [ ]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Kyiv', 'Vienna', 'Birmingham', ..., 'Berlin', 'Rome', 'Milan'],
       ['Sofia', 'Istanbul', 'Birmingham', ..., 'Paris', 'Amsterdam',
        'Brussels'],
       ['Berlin', 'London', 'Kyiv', ..., 'Paris', 'Milan', 'Birmingham'],
       ...,
       ['Warsaw', 'Kyiv', 'Sofia', ..., 'Barcelona', 'Paris', 'Brussels'],
       ['Milan', 'Brussels', 'Sofia', ..., 'Paris', 'Prague', 'Kyiv'],
       ['Paris', 'Munich', 'Istanbul', ..., 'Prague', 'Bucharest',
        'Brussels']], dtype='<U10')

In [ ]:
# 2. Evaluation of the fitness

#individual solution
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

#All solutions
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([ 794.27256038,  935.89342498,  910.51434531, 1062.72787561,
        965.15661228, 1194.653385  ,  897.94770377,  889.46612054,
        884.16694003,  844.98597528,  909.6641712 ,  912.26544009,
        890.69315265,  968.71569793, 1093.96673829,  866.85022628,
       1010.86647434,  996.9621264 ,  930.48922349,  969.14273722,
       1032.43187159,  997.8341618 , 1009.19894872,  977.89105002,
        844.69777731,  956.72542078,  939.58468439, 1110.13941701,
        856.32189684,  944.88543278, 1135.69246235,  909.36656986,
       1011.7429743 , 1002.28627519,  726.66103025,  805.72103551,
        918.74831259,  987.04916343,  910.4178529 , 1017.71366672,
        995.77244689,  832.2793591 , 1053.9982397 ,  847.16599365,
        997.69065441,  942.7143773 , 1056.1506807 ,  985.11912056,
       1148.17425822,  857.11726897,  924.46341796, 1067.42396987,
        857.47309396,  854.91583006,  874.069375  ,  988.8886635 ,
       1018.04541056,  863.56372166,  784.73145073,  936.93431

In [14]:
# 3. Selecting the progenitors
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])
    
progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['Budapest', 'Munich', 'Rome', 'London', 'Berlin', 'Warsaw',
       'Paris', 'Brussels', 'Prague', 'Bucharest', 'Moscow', 'Sofia',
       'Amsterdam', 'Istanbul', 'Minsk', 'Vienna', 'Birmingham', 'Milan',
       'Kyiv', 'Barcelona'], dtype='<U10')

In [ ]:
# Pairs crossover
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
# Finding pairs of mates
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Budapest', 'Munich', 'Rome', 'London', 'Berlin', 'Birmingham',
       'Prague', 'Bucharest', 'Vienna', 'Moscow', 'Kyiv', 'Minsk',
       'Paris', 'Istanbul', 'Amsterdam', 'Milan', 'Warsaw', 'Brussels',
       'Barcelona', 'Sofia'], dtype='<U10')

In [ ]:
#Offspring production
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
# New populaiton generation
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['Berlin', 'Amsterdam', 'Rome', 'London', 'Warsaw', 'Birmingham',
       'Prague', 'Paris', 'Istanbul', 'Moscow', 'Kyiv', 'Minsk', 'Milan',
       'Vienna', 'Munich', 'Bucharest', 'Budapest', 'Brussels',
       'Barcelona', 'Sofia'], dtype='<U10')

In [ ]:
# Everything put together
best_solution = [-1,np.inf,np.array([])]
for i in range(10000):
    if i%500==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 764.5484926857778 941.8184283940618 12/12/22 02:19
500 744.5441375926367 957.0871721934999 12/12/22 02:19
1000 757.6368051423085 945.2585015131015 12/12/22 02:20
1500 653.9535846641174 943.1947515711746 12/12/22 02:20
2000 731.3423244115378 937.2110200183856 12/12/22 02:20
2500 697.861101965572 938.9123425905391 12/12/22 02:20
3000 726.9510283927755 947.3564772724988 12/12/22 02:20
3500 734.2081066815861 929.2722380136045 12/12/22 02:21
4000 751.9265969957316 944.1697472292835 12/12/22 02:21
4500 719.8233294672007 934.3689712004854 12/12/22 02:21
5000 688.2263682338115 942.134253697959 12/12/22 02:21
5500 764.1361763809201 943.0847780875134 12/12/22 02:22
6000 669.2022851420917 933.2708442318628 12/12/22 02:22
6500 700.8708029709074 940.8891661921309 12/12/22 02:22
7000 681.5437636364618 940.1904279036518 12/12/22 02:22
7500 711.7861018244391 941.8354807122407 12/12/22 02:22
8000 692.0154070450377 940.7516306786205 12/12/22 02:23
8500 735.6403878385339 946.1917555211243 12/12/22 02:2

In [ ]:
best_solution

[2312,
 482.0251974400215,
 array([['Istanbul', 'Budapest', 'Amsterdam', 'London', 'Munich',
         'Brussels', 'Barcelona', 'Bucharest', 'Warsaw', 'Minsk', 'Rome',
         'Kyiv', 'Milan', 'Moscow', 'Sofia', 'Paris', 'Vienna',
         'Birmingham', 'Berlin', 'Prague']], dtype='<U10')]